In [1]:
!pip install qiskit
!pip install qiskit-aer
!pip install qiskit[visualization]

In [2]:
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import math
from fractions import Fraction

In [3]:
def continued_fraction( x, max_denominator ):
    frac = Fraction( x ).limit_denominator( max_denominator )
    return frac.denominator

In [4]:
def quantum_order_finding( a, N ):
    n = int( math.ceil( math.log2( N ) ) )
    
    qc = QuantumCircuit( 2*n + n, 2*n )
    qc.x( 2*n + n - 1 )
    qc.h( range( 2*n ) )
    
    # Inverse QFT on upper register (simplified)
    for qubit in range( 2*n // 2):
        qc.swap( qubit, 2*n - qubit - 1 )
    for j in range( 2*n ):
        for m in range( j ):
            qc.cp( -math.pi / float( 2 ** ( j - m ) ), m, j )
        qc.h( j )
        
    qc.measure( range( 2*n ), range( 2*n ) )
    
    simulator = AerSimulator()
    job = simulator.run( qc, shots = 1024 )
    result = job.result()
    counts = result.get_counts()
    
    measured = max( counts, key = counts.get )
    decimal = int( measured, 2 )
    
    phase = decimal / ( 2 ** ( 2 * n ) )
    r = continued_fraction( phase, N )
    
    return r

In [5]:
def find_factors( N ):
    for a in range( 2, N ):
        if math.gcd( a, N ) != 1:
            factor = math.gcd(a, N)
            print( f"Found non-trivial factor {factor} by gcd with a={a}" )
            return factor
    
        print( f"Trying a = {a}" )
        r = quantum_order_finding (a, N )
        print( f"Estimated order r = {r}" )
    
        if r is None or r % 2 != 0:
            print( f"r = {r} is odd or None; trying next a" )
            continue
    
        x = pow( a, r // 2, N )
        
        if x == N - 1:
            print( "x ≡ -1 mod N; trying next a" )
            continue
    
        factor1 = math.gcd( x - 1, N )
        factor2 = math.gcd( x + 1, N )
    
        print( f"Possible factors: {factor1}, {factor2}" )
        if factor1 != 1 and factor1 != N:
            print( f"Non-trivial factor found: {factor1}" )
            return factor1

        if factor2 != 1 and factor2 != N:
            print (f"Non-trivial factor found: {factor2}" )
            return factor2

    print("Failed to find factors with all candidates")
    return None

In [ ]:
N = 11235
factor = find_factors(N)
print(f"Result: factor of {N} is {factor}")

Trying a = 2
Estimated order r = 1
r = 1 is odd or None; trying next a
Found non-trivial factor 3 by gcd with a=3
Result: factor of 11235 is 3
